In [1]:
import os

from pathling import PathlingContext
from pyspark.sql.functions import lit

MIMIC_FHIR_PATH="/Users/szu004/dev/mimic-vi-fhir/target/mimic4-demo-pqt"

In [2]:
pc = PathlingContext.create()

:: loading settings :: url = jar:file:/Users/szu004/miniconda3/envs/sof_mimic/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/szu004/.ivy2/cache
The jars for the packages stored in: /Users/szu004/.ivy2/jars
au.csiro.pathling#library-runtime added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c63121ff-462a-4643-b887-b98d81a3b366;1.0
	confs: [default]
	found au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT in local-m2-cache
	found io.delta#delta-spark_2.12;3.2.0 in local-m2-cache
	found io.delta#delta-storage;3.2.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.9.3 in local-m2-cache
	found org.apache.hadoop#hadoop-aws;3.3.4 in local-m2-cache
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in local-m2-cache
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in local-m2-cache
:: resolution report :: resolve 120ms :: artifacts dl 6ms
	:: modules in use:
	au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT from local-m2-cache in [default]
	co

2024-08-27 16:08:33 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-08-27 16:08:35 INFO  Version:21 - HV000001: Hibernate Validator null
2024-08-27 16:08:35 DEBUG AbstractConfigurationImpl:176 - Setting custom MessageInterpolator of type au.csiro.pathling.shaded.org.hibernate.validator.messageinterpolation.ParameterMessageInterpolator
2024-08-27 16:08:35 DEBUG ValidationXmlParser:122 - Trying to load META-INF/validation.xml for XML based Validator configuration.
2024-08-27 16:08:35 DEBUG ResourceLoaderHelper:61 - Trying to load META-INF/validation.xml via TCCL
2024-08-27 16:08:35 DEBUG ResourceLoaderHelper:67 - Trying to load META-INF/validation.xml via Hibernate Validator's class loader
2024-08-27 16:08:35 DEBUG ValidationXmlParser:129 - No META-INF/validation.xml found. Using annotation based configuration only.
2024-08-27 16:08:35 DEBUG TraversableResolvers:61 - Cannot find jakarta.persistence.Per

In [3]:
data = pc.read.parquet(MIMIC_FHIR_PATH)
data.read('Patient').show()

2024-08-27 16:08:36 WARN  SparkStringUtils:72 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+-------------+--------+----+--------------------+------+--------------------+-------+------+----------+---------------+----------------+-------+--------------------+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+----+----------+--------------------+
|                  id|        id_versioned|                meta|implicitRules|language|text|          identifier|active|                name|telecom|gender| birthDate|deceasedBoolean|deceasedDateTime|address|       maritalStatus|multipleBirthBoolean|multipleBirthInteger|photo|contact|       communication|generalPractitioner|managingOrganization|link|      _fid|          _extension|
+--------------------+--------------------+--------------------+-------------+--------+----+--------------------+------+--------------------+-------+------+----------+---------------+----------------+-------+--------------------+-

In [16]:
ptn_view = data.view('Patient', 
    select = [
        {
            "column": [
                {
                    "name": "id",
                    "path": "id",
                    "collection": False
                },
                {
                    "name": "gender",
                    "path": "gender",
                    "collection": False
                },
                {
                    "name": "race",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('text').value.ofType(string)",
                    "collection": False
                },
                {
                    "name": "ethnicity",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity').extension('text').value.ofType(string)",
                    "collection": False
                },
                
            ]
        }
    ]
)

ptn_view.show()

+--------------------+------+--------------------+--------------------+
|                  id|gender|                race|           ethnicity|
+--------------------+------+--------------------+--------------------+
|28dcf33b-0c52-587...|female|               White|Not Hispanic or L...|
|74a2fd87-885b-5ec...|female|               White|Not Hispanic or L...|
|51d2190c-cc46-56c...|  male|               White|  Hispanic or Latino|
|c4c29979-f2f5-56d...|  male|               White|  Hispanic or Latino|
|b410dd44-7d65-56f...|  male|               White|Not Hispanic or L...|
|a3a12d01-dc21-565...|  male|               White|Not Hispanic or L...|
|807d2a03-9c57-55e...|female|             unknown|                NULL|
|77e10fd0-6a1c-554...|female|Black or African ...|Not Hispanic or L...|
|aeda7084-be35-510...|female|               White|  Hispanic or Latino|
|5f3dcdb5-bd27-58f...|  male|               White|Not Hispanic or L...|
|2955c958-192a-50e...|female|Black or African ...|Not Hispanic o

In [14]:
ptn_view.select('race').distinct().show()

+--------------------+
|                race|
+--------------------+
|   asked but unknown|
|             unknown|
|               White|
|Black or African ...|
+--------------------+

